In [2]:
import json
import requests
from datetime import datetime, timedelta, time, date

# staging token
token = "your token"
token_old = "your token"


search_url = "https://api.intercom.io/conversations"
contant_search_url = "https://api.intercom.io/contacts/search"
admin_search_url = "https://api.intercom.io/admins"
create_message_url = "https://api.intercom.io/messages"
create_conversation_url = "https://api.intercom.io/conversations"
find_conv_url =  "https://api.intercom.io/conversations/search"
last_conv_url = "https://api.intercom.io/conversations/last/reply"


login = {
    "Accept": "application/json",
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

login_old = {
    "Accept": "application/json",
    "Authorization": f"Bearer {token_old}",
    "Content-Type": "application/json"
}


def find_intercom_id(user_id):
    find_id_params = {
        "query": {
            "field": "external_id",
            "operator": "=",
            "value": f"{user_id}"
        }
    }
    response = requests.post(url=contant_search_url, headers=login, json=find_id_params)
    data = response.json()
    intercom_id = data["data"][0]["id"]
    return intercom_id


def admin_finder(name_surname):
    response = requests.get(url=admin_search_url, headers=login).json()
    for i in response["admins"]:
        if f"{name_surname}" in i["name"]:
            return i["id"]

def snooze_conversation(conv_id,admin_id):
    snooze_conv_url = f"https://api.intercom.io/conversations/{conv_id}/reply"
    snooze_params = {
        "message_type":"snoozed",
        "admin_id": f"{admin_id}",
        "snoozed_until": '1653831544'
    }
    response = requests.post(url=snooze_conv_url,headers=login,data=snooze_params)
    #return response

def assing_conversation(conv_id, assign_id):
    assing_conversation_url = f"https://api.intercom.io/conversations/{conv_id}/parts"
    assing_params = {
        "type": "admin",
        "admin_id": assign_id,
        "assignee_id": assign_id,
        "message_type": "assignment",
    }
    response = requests.post(url=assing_conversation_url, headers=login, json=assing_params)
    return assign_id

def add_note(conv_id,admin_id):
    add_note_url = f"https://api.intercom.io/conversations/{conv_id}/reply"
    add_note_data = {
        "message_type": "note",
        "type": "admin",
        "admin_id": f"{admin_id}",
        "body": f"{conv_id}'ye not/yorum eklendi.."}
    response = requests.post(url=add_note_url, headers=login, data=add_note_data)
    #return response

def send_message(conv_id,admin_id):
    add_note_url = f"https://api.intercom.io/conversations/{conv_id}/reply"
    add_note_data = {
        "message_type": "comment",
        "type": "admin",
        "admin_id": f"{admin_id}",
        "body": f"{conv_id}'ye {admin_id}'den mesaj gönderildi... ",
    }
    response = requests.post(url=add_note_url, headers=login, json=add_note_data).json()
    return response

def create_message(admin_id,user_id):
    create_message_params = {
      "message_type": "inapp",
      "body": "Deneme text",
      "from": {"type": "admin", "id": admin_id},
      "to": {"type": "contact", "id": user_id},
    }
    response = requests.post(url=create_message_url, headers=login_old, json=create_message_params)
    #return response.json()

def create_conversation(admin_id, user_id):
    create_conv_params = {
        "message_type": "inapp",
        "from": {"type": "admin", "id": admin_id},
        "to": {"type": "contact", "id": user_id},
        "body": f"{user_id}'ye mesaj oluşturuldu. Satıcı fatura göndermemişsin!"}
    response = requests.post(url=create_conversation_url, headers=login, json=create_conv_params).json()
    #return response


def find_last_conv(user_id, admin_id):
    last_conv_param = {
        "intercom_user_id": user_id,
        "body": "Deneme",
        "type": "admin",
        "admin_id": admin_id,
        "message_type": "note"
    }
    response = requests.post(url=last_conv_url, headers=login_old, json=last_conv_param).json()
    return response["id"]


#Temsilci id ve conv id ile eşleşen konuşmayı şimdiden 7 gün sonraya kadar snooze et
def snooze_conversation_param(conv_id,admin_id,delay_day=int):
    week = datetime.today() + timedelta(days=delay_day)
    week_timestamp = int(datetime.timestamp(week))
    snooze_conv_url = f"https://api.intercom.io/conversations/{conv_id}/reply"
    snooze_params = {
        "message_type":"snoozed",
        "admin_id": f"{admin_id}",
        "snoozed_until": week_timestamp,
    }
    response = requests.post(url=snooze_conv_url,headers=login,json=snooze_params).json()
    return response

In [2]:
import gspread
GS_CREDENTIAL_FILENAME = "gs_credential.json"  # İndirilen json dosyası yolu
GS_FILENAME = "Example"  # Google Sheet dosyası adı
GS_SHEET_INDEX = 0  # Sheet2'e erişim için
gc = gspread.service_account(GS_CREDENTIAL_FILENAME)
wks = gc.open(GS_FILENAME).get_worksheet(GS_SHEET_INDEX)

##Amaç: Yazılan değerin en son nerde kalmış onun üstünden devam ettir
#for i in range(1,5):
#    a = len(wks.get_values('A:A'))
#    wks.update_acell(f"A{a+1}", "Conversation_id")
#    i+=1

In [3]:
month = datetime.today() - timedelta(days=30)
month_timestamp = int(datetime.timestamp(month))

today = datetime.today()
today_timestamp = int(datetime.timestamp(today))

In [8]:
import requests
from datetime import datetime,timedelta
import calendar, time
import re
import pandas as pd

IntercomUrl = 'https://api.intercom.io/conversations/search'
token = "your_token"  # Eski token
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {token}"}

# Son 5 dakikada olan conservation
#now = int(datetime.now().timestamp())
#five = datetime.now() - timedelta(minutes=15)
#five_unix = int(time.mktime(five.timetuple()))

SearchQuery = {
    "query": {
        "operator": "AND",
        "value": [
            {
                "field": "created_at",
                "operator": "<",
                "value": today_timestamp
            },
            {
                "field": "created_at",
                "operator": ">",
                "value": month_timestamp
            },
            {
                "field": "state",
                "operator": "=",
                "value": "open"
            }

        ]
    }
}

# İstek atmak istediğimiz link için fonksiyon
conversations_id   = []
def get_data():
    response = requests.post(IntercomUrl, headers=headers, json=SearchQuery)
    data = response.json()
    for k in data['conversations']:
        print(k)
        #if "UTS" in k['source']['body']:
            #first_intercom_conv_id = k['id']
            #conversations_id.append(first_intercom_conv_id)
            
            
            
#df = pd.DataFrame(list(zip(conversations_id)),
               #columns =['conversations_id'])
#df.head()

In [5]:
import requests
from datetime import datetime,timedelta
import calendar, time
import re

IntercomUrl = 'https://api.intercom.io/conversations/search'
token = "your_token"  # Eski token
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {token}"}

# Son 5 dakikada olan conservation
#now = int(datetime.now().timestamp())
#five = datetime.now() - timedelta(minutes=15)
#five_unix = int(time.mktime(five.timetuple()))

SearchQuery = {
    "query": {
        "operator": "AND",
        "value": [
            {
                "field": "created_at",
                "operator": "<",
                "value": today_timestamp
            },
            {
                "field": "created_at",
                "operator": ">",
                "value": month_timestamp
            },
            {
                "field": "state",
                "operator": "=",
                "value": "open"
            }

        ]
    }
}


# İstek atmak istediğimiz link için fonksiyon
def get_data():
    response = requests.post(IntercomUrl, headers=headers, json=SearchQuery)
    data = response.json()
    for k in data['conversations']:
        if "Fatura Gelmedi" in k['source']['body']:
            first_intercom_conv_id = k['id']
            buyer_intercom_name = k['source']['author']['name']
            print(first_intercom_conv_id)
            print("İlk conv id")
            with open('control.txt',"a+") as txt:
                txt.seek(0)
                lines = [line.rstrip() for line in txt.readlines()]
                if first_intercom_conv_id not in lines:
                    txt.writelines([first_intercom_conv_id + '\n'])
                    url = k["source"]['url']
                    
                    a = len(wks.get_values('A:A'))
                    wks.update_acell(f"A{a+1}", first_intercom_conv_id) # İlk konuşma conv_id
                    
                    b = len(wks.get_values('B:B'))
                    wks.update_acell(f"B{b+1}", assing_conversation(conv_id=first_intercom_conv_id,assign_id=admin_finder("Mikail Dogruer")))
                    print("İlk pencere Mikai'e assign edildi.")
                    seller_id = 999999999999
                    # create conversation from admin to user, buraya içerik eklemesi yapılacak
                    create_conversation(admin_id=admin_finder("Mikail Dogruer"),user_id=find_intercom_id(seller_id))
                    print("Satıcıya mesaj oluşturuldu")
                    seller_intercom = find_last_conv(find_intercom_id(seller_id),admin_finder("Mikail Dogruer"))
                    seller_window_id = seller_intercom
                    print("Satıcı penceresinin id'si")
                    print(seller_window_id)
                    assing_conversation(seller_window_id,admin_finder("Mikail Dogruer"))
                    print("Pencere Mikail'e assign edildi")
                    # notları ekle, not eklenecek datayı değiştiricez
                    add_note(seller_window_id,admin_finder("Mikail Dogruer"))
                    add_note(first_intercom_conv_id, admin_finder("Mikail Dogruer"))
                    print("Notlar eklendi")
                    # .send_message(conv_id=seller_window_id,admin_id=.admin_finder("Mikail Dogruer"))
                    # print("Satıcıya mesaj yollandı")
                    send_message(conv_id=first_intercom_conv_id,admin_id=admin_finder("Mikail Dogruer"))
                    now_message = datetime.now()
                    dt_string = now_message.strftime("%d/%m/%Y %H:%M:%S")
                    c = len(wks.get_values('C:C'))
                    wks.update_acell(f"C{c+1}", dt_string) #Mesaj yollandığı an ki saat 
                    print("Alıcıya mesaj yollandı")
                    # snooze
                    snooze_conversation_param(first_intercom_conv_id,admin_finder("Mikail Dogruer"),1)
                    snooze_conversation_param(seller_window_id, admin_finder("Mikail Dogruer"), 7)
                    print("Snoozelar yapılıyor.")
                else:
                    print('Daha önce olan conversation')
                    pass
get_data()

135887500000275
İlk conv id
İlk pencere Mikai'e assign edildi.
Satıcıya mesaj oluşturuldu
Satıcı penceresinin id'si
135887500000276
Pencere Mikail'e assign edildi
Notlar eklendi
Alıcıya mesaj yollandı
Snoozelar yapılıyor.
